#### Montaje Azure Data Lake mediante service ppal.
- 1. Obtener datos de acceso desde Key Vault.
- 2. Configurar Spark con Client ID, Tenant ID y Secret.
- 3. Utilizar el método "mount" para montar el almacenamiento.
- 4. Explorar otras utilidades del sistema de archivos.

##### Variables

In [0]:
client_id=dbutils.secrets.get("db-secret-scope", "Databricks-dev-clientid")
tenant_id=dbutils.secrets.get("db-secret-scope", "azure-tenant-id")
client_secret=dbutils.secrets.get("db-secret-scope", "Databricks-dev-secret")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

In [0]:
spark.conf.set("fs.azure.account.auth.type.moviehistorydev.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.moviehistorydev.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.moviehistorydev.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.moviehistorydev.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.moviehistorydev.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://demo@moviehistorydev.dfs.core.windows.net/",
  mount_point = "/mnt/moviehistory/demo",
  extra_configs = configs)

In [0]:
display(dbutils.fs.ls("mnt/moviehistory/demo"))

In [0]:
df_movie = spark.read.format("csv").option("header", "true").load("/mnt/moviehistory/demo/movie.csv")
display(df_movie)

In [0]:
display(dbutils.fs.mounts())

In [0]:
dbutils.fs.unmount("/mnt/moviehistory/demo")